#### data preprocessing

In [1]:
import pandas as pd

In [ ]:
%%bash 

cut -d$'\t' -f1-4 spidex_public_noncommercial_v1_0.tab.abs_1.5 > spidex_cut.txt

In [16]:
with open('spidex_cut.txt', 'r') as f:
    with open('spidex_cut.vcf', 'w') as v:
        for i in f.readlines():
            l = i.split('\t')
            l.insert(2, '.')
            v.write('\t'.join(l))

In [8]:
%%bash

sed -i '1d;2i #CHROM\tPOS\tID\tREF\tALT' spidex_cut.vcf 

In [9]:
df = pd.read_table('spidex_cut.vcf')

In [11]:
gp = df.groupby('#CHROM')
for n,g in gp:
    g.to_csv('./spidex_tmp_dir/spidex_%s.vcf'%(n), index=False, sep='\t')

#### mmsplice annotation

In [ ]:
%%bash 

docker run -v ~/biodatabases:/biodatabases -v /data:/data -it ac604aad4bf7 bash

In [ ]:
#bash in docker
%%bash

parallel -j 6 '/home/bioapps/ensembl-vep/vep -i /data/mmsplic_tmp/{}.vcf --plugin MMSplice --vcf --force --assembly GRCh37 --cache --port 3337 -o /data/mmsplic_tmp/output/{}_mmsplice.vcf —pick' :::: <(ls /data/mmsplic_tmp | grep vcf |cut -d'.' -f1)

#### SpliceAI annotation

In [ ]:
%%bash

parallel -j 3 'spliceai -I /data/mmsplic_tmp/{}.vcf -O /data/mmsplic_tmp/splicai_output/{}_spliceai.vcf -R ~/biodatabases/human_g1k_v37.fasta -A grch37' :::: <(ls /data/mmsplic_tmp | grep vcf |cut -d'.' -f1)